In [2]:
from _121002_utils import generate_date_ranges


In [3]:
my_ranges = generate_date_ranges(2023,2025, 'month')
my_ranges

[('2023-01-01', '2023-01-31'),
 ('2023-02-01', '2023-02-28'),
 ('2023-03-01', '2023-03-31'),
 ('2023-04-01', '2023-04-30'),
 ('2023-05-01', '2023-05-31'),
 ('2023-06-01', '2023-06-30'),
 ('2023-07-01', '2023-07-31'),
 ('2023-08-01', '2023-08-31'),
 ('2023-09-01', '2023-09-30'),
 ('2023-10-01', '2023-10-31'),
 ('2023-11-01', '2023-11-30'),
 ('2023-12-01', '2023-12-31'),
 ('2024-01-01', '2024-01-31'),
 ('2024-02-01', '2024-02-29'),
 ('2024-03-01', '2024-03-31'),
 ('2024-04-01', '2024-04-30'),
 ('2024-05-01', '2024-05-31'),
 ('2024-06-01', '2024-06-30'),
 ('2024-07-01', '2024-07-31'),
 ('2024-08-01', '2024-08-31'),
 ('2024-09-01', '2024-09-30'),
 ('2024-10-01', '2024-10-31'),
 ('2024-11-01', '2024-11-30'),
 ('2024-12-01', '2024-12-31'),
 ('2025-01-01', '2025-01-31'),
 ('2025-02-01', '2025-02-13')]